In [1]:
# !pip install torch --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install transformers
# !pip install datasets
# !pip install deepspeed
# !pip install mpi4py
# !pip install accelerate

In [1]:
import os, re
import torch
import numpy as np

from datasets import load_dataset
from transformers import BloomTokenizerFast, BloomForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments

/home/gordon/anaconda3/envs/pinxi/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_path = os.getcwd()
model_root_path = os.path.join(root_path, "models")
data_root_path = os.path.join(root_path, "data")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [3]:
model_name = "bloom-1b7"
model_path = os.path.join(model_root_path, model_name)

In [4]:
model = BloomForCausalLM.from_pretrained(model_path).to(device)

In [5]:
# model = BloomForCausalLM.from_pretrained(f"bigscience/{model_name}").to(device)
# model.save_pretrained(model_path)

In [6]:
data_file = "lm_generated_data.jsonl"
dataset_raw = load_dataset("json", data_files=data_file, data_dir=data_root_path)
dataset_split = dataset_raw["train"].train_test_split(test_size=0.1)

Using custom data configuration default-8b59437041d29b59
Found cached dataset json (/home/gordon/.cache/huggingface/datasets/json/default-8b59437041d29b59/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 272.68it/s]


In [7]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['email_prompt', 'email', 'summary'],
        num_rows: 3600
    })
    test: Dataset({
        features: ['email_prompt', 'email', 'summary'],
        num_rows: 400
    })
})

In [8]:
tokenizer = BloomTokenizerFast.from_pretrained(f"bigscience/{model_name}")

In [9]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [10]:
def tokenize_data(examples):
    relevant_texts = zip(examples["email"], examples["summary"])
    concat_texts = [t[0] + "\n\n===\n\n" + t[1] + "\nEND" for t in relevant_texts]
    results = tokenizer(concat_texts, padding="max_length", max_length=400, truncation=True)
    results["labels"] = results["input_ids"].copy()
    return results

In [11]:
dataset_tokenized = dataset_split.map(tokenize_data, batched=True, remove_columns=dataset_raw["train"].column_names)

Parameter 'function'=<function tokenize_data at 0x7ff8cf5c0170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
  0%|                                                                                     | 0/1 [00:00<?, ?ba/s]


In [12]:
dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3600
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 400
    })
})

In [13]:
seen = set()
for x in dataset_tokenized["train"]:
    y = len(x["input_ids"])
    if y not in seen:
        print(y)
        seen.add(y)

400


In [14]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [15]:
model_save_path = os.path.join(model_root_path, f"finetuned-{model_name}-003-e2")
ds_config = "ds_config_zero2.json"

training_args = TrainingArguments(
    output_dir=model_save_path,
    num_train_epochs=2,
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=1e-3,
    warmup_ratio=0.1,
    deepspeed=ds_config,
    evaluation_strategy="steps",
    save_strategy="epoch",
    logging_steps=25,
    eval_steps=25,
)

[2022-10-22 23:20:54,285] [INFO] [comm.py:618:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...
[2022-10-22 23:20:54,784] [INFO] [comm.py:675:mpi_discovery] Discovered MPI settings of world_rank=0, local_rank=0, world_size=1, master_addr=192.168.1.116, master_port=29500
[2022-10-22 23:20:54,787] [INFO] [comm.py:635:init_distributed] Initializing TorchBackend in DeepSpeed with backend nccl


In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["test"],
    data_collator=data_collator,
)

In [17]:
trainer.train()
model.save_pretrained(model_save_path)

[2022-10-22 23:20:54,826] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed info: version=0.7.3, git-hash=unknown, git-branch=unknown
[2022-10-22 23:20:54,898] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
Using /home/gordon/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/gordon/.cache/torch_extensions/py37_cu113/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module cpu_adam...
Time to load cpu_adam op: 2.898439407348633 seconds
[2022-10-22 23:21:00,053] [INFO] [logging.py:68:log_dist] [Rank 0] Using DeepSpeed Optimizer param name adamw as basic optimizer
[2022-10-22 23:21:00,067] [INFO] [logging.py:68:log_dist] [Rank 0] DeepSpeed Basic Optimizer = {basic_optimizer.__class__.__name__}
[2022-10-22 23

[2022-10-22 23:21:13,952] [INFO] [config.py:991:print]   eigenvalue_tol ............... 0.01
[2022-10-22 23:21:13,952] [INFO] [config.py:991:print]   eigenvalue_verbose ........... False
[2022-10-22 23:21:13,953] [INFO] [config.py:991:print]   elasticity_enabled ........... False
[2022-10-22 23:21:13,953] [INFO] [config.py:991:print]   flops_profiler_config ........ {
    "enabled": false, 
    "profile_step": 1, 
    "module_depth": -1, 
    "top_modules": 1, 
    "detailed": true, 
    "output_file": null
}
[2022-10-22 23:21:13,954] [INFO] [config.py:991:print]   fp16_auto_cast ............... None
[2022-10-22 23:21:13,955] [INFO] [config.py:991:print]   fp16_enabled ................. False
[2022-10-22 23:21:13,955] [INFO] [config.py:991:print]   fp16_master_weights_and_gradients  False
[2022-10-22 23:21:13,956] [INFO] [config.py:991:print]   global_rank .................. 0
[2022-10-22 23:21:13,956] [INFO] [config.py:991:print]   gradient_accumulation_steps .. 4
[2022-10-22 23:21:13

***** Running training *****
  Num examples = 3600
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 4
  Total optimization steps = 450
You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Using /home/gordon/.cache/torch_extensions/py37_cu113 as PyTorch extensions root...
No modifications detected for re-loaded extension module utils, skipping build step...
Loading extension module utils...
Time to load utils op: 0.0013315677642822266 seconds
Adam Optimizer #0 is created with AVX2 arithmetic capability.
Config: alpha=0.000100, betas=(0.900000, 0.999000), weight_decay=0.001000, adam_w=1


Step,Training Loss,Validation Loss
25,2.022200,1.646748
50,1.606000,1.558760
75,1.551900,1.492738
100,1.527000,1.490860
125,1.487900,1.437143
150,1.413600,1.443047
175,1.422100,1.400741
200,1.379400,1.355497
225,1.365600,1.365960
250,1.080300,1.343960


...

***** Running Evaluation *****
  Num examples = 400
  Batch size = 8


[2022-10-23 01:03:02,075] [INFO] [timer.py:207:stop] 0/1710, RunningAvgSamplesPerSec=1.3616408922688963, CurrSamplesPerSec=1.5938916648338939, MemAllocated=8.33GB, MaxMemAllocated=19.94GB
[2022-10-23 01:03:32,805] [INFO] [logging.py:68:log_dist] [Rank 0] step=430, skipped=0, lr=[5.185185185185185e-06], mom=[[0.9, 0.999]]
[2022-10-23 01:03:32,852] [INFO] [timer.py:207:stop] 0/1720, RunningAvgSamplesPerSec=1.3612745198621452, CurrSamplesPerSec=0.8317874046787228, MemAllocated=8.33GB, MaxMemAllocated=19.94GB
[2022-10-23 01:04:00,791] [INFO] [timer.py:207:stop] 0/1730, RunningAvgSamplesPerSec=1.3616726011883415, CurrSamplesPerSec=1.603398775537861, MemAllocated=8.33GB, MaxMemAllocated=19.94GB
[2022-10-23 01:04:31,520] [INFO] [timer.py:207:stop] 0/1740, RunningAvgSamplesPerSec=1.3613227818636409, CurrSamplesPerSec=0.8353402584806334, MemAllocated=8.33GB, MaxMemAllocated=19.94GB
[2022-10-23 01:04:59,406] [INFO] [timer.py:207:stop] 0/1750, RunningAvgSamplesPerSec=1.3617300946322934, CurrSampl

***** Running Evaluation *****
  Num examples = 400
  Batch size = 8
Saving model checkpoint to /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/checkpoint-450
Configuration saved in /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/checkpoint-450/config.json
Model weights saved in /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/checkpoint-450/pytorch_model.bin


[2022-10-23 01:08:40,616] [INFO] [logging.py:68:log_dist] [Rank 0] [Torch] Checkpoint global_step450 is begin to save!
[2022-10-23 01:08:40,620] [INFO] [logging.py:68:log_dist] [Rank 0] Saving model checkpoint: /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/checkpoint-450/global_step450/mp_rank_00_model_states.pt
[2022-10-23 01:08:40,621] [INFO] [torch_checkpoint_engine.py:15:save] [Torch] Saving /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/checkpoint-450/global_step450/mp_rank_00_model_states.pt...
[2022-10-23 01:08:53,210] [INFO] [torch_checkpoint_engine.py:17:save] [Torch] Saved /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/checkpoint-450/global_step450/mp_rank_00_model_states.pt.
[2022-10-23 01:08:53,211] [INFO] [torch_checkpoint_engine.py:15:save] [Torch] Saving /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/checkpoint-450/global_step450/zero_pp_rank_0_mp_rank_00_optim_states.pt...
[2022-10-23 01:09:21,404] [INFO] [torch_check



Training completed. Do not forget to share your model on huggingface.co/models =)


Configuration saved in /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/config.json
Model weights saved in /home/gordon/pinxi/cs4248/models/finetuned-bloom-1b7-003-e2/pytorch_model.bin


In [23]:
sample_prompt = """Hey John,

Great getting to know you and your team. I'm excited to be part of this amazing department and to be working with you!

In the meantime, could you take a look at the minutes of the budget review discussed today? I'd need to know by Thursday if that's possible.

Also, could you share any progress you have had on the marketing review? I will need to update the management team later. Also will need you to setup a Zoom call tomorrow 2pm for this meeting. 

Thanks.
Peter

===

"""

input_ids = tokenizer.encode(sample_prompt, return_tensors="pt").to(device)
pred_ids = model.generate(
    input_ids,
    max_length = len(input_ids[0]) + 1000,
)
pred_text = tokenizer.decode(pred_ids[0])
start_idx = pred_text.find("===\n\n") + 5
end_idx = pred_text.find("\nEND")
pred_output = pred_text[start_idx:end_idx]
print(pred_output)

- review the attached document by this Thursday
- schedule an online meeting on at 2 pm for tomorrow
- give an update
